In [3]:
import re
import xml.etree.ElementTree as ET
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer, PorterStemmer
from nltk.corpus import stopwords
import nltk
import glob

In [2]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Andreas\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Andreas\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
import xml.etree.ElementTree as ET

def extract_text_from_abstract(path):
    tree = ET.parse(path)
    root = tree.getroot()
    ns = {'ns0': 'http://www.tei-c.org/ns/1.0'}

    # Find the <abstract> element
    abstract = root.find(".//ns0:abstract", ns)
    
    if abstract is not None:
        # Extract all <p> tags within <div> inside <abstract>
        paragraphs = abstract.findall(".//ns0:div//ns0:p", ns)

        # Extract and concatenate text content from all <p> tags
        extracted_text = "\n".join([p.text.strip() for p in paragraphs if p.text])

        return extracted_text
    else:
        return None

In [4]:
files = []
for file in glob.glob("../../resources/XML_CEUR/*/*.tei.xml"):
    files.append(file)
for file in glob.glob("../../resources/XML_CEUR/*/*/*.tei.xml"):
    files.append(file)
    

In [5]:
corpus_text = []

In [6]:
for i in files:
    string = extract_text_from_abstract(i)
    corpus_text.append(string)


In [7]:
stemmer = SnowballStemmer('english')


In [22]:
import string

def custom_tokenizer(text):
    from nltk.corpus import stopwords
    text = text.translate(str.maketrans('', '', string.punctuation))

    stopwords_labs_tracks = [ "Adhoc", "AToMiC", "Blog", "CENTRE","Chemical", "Chinese",
                            "Clinical Decision Support", "Clinical Trials", "Common Core", "Complex Answer Retrieval", "Contextual Suggestion", "Conversational Assistance",
                            "CrisisFACTs", "Cross-Language", "Crowdsourcing", "Decision", "Deep Learning", "Dynamic Domain", "Enterprise", "Fair Ranking", "Entity Track",
                            "Federated Web Search","Filtering", "Genomics", "HARD", "Health Misinformation", "Incident Streams", "Interactive", "Knowledge Base Acceleration",
                            "Legal", "LiveQA", "Medical", "Microblog", "Million Query", "NeuCLIR", "News", "Novelty", "OpenSearch", "Podcast", "Precision Medicine",
                            "Query Track", "Question Answering", "Real-time Summarization", "Relevance Feedback", "Robust", "Session", "Spam", "Spoken Document Retrieval",
                            "Tasks Track", "Temporal Summarization", "Terabyte", "Total Recall", "Video Track", "Web Track",
                            "German Indexing and Retrieval Testdatabase", "GIRT", "Cross-Language Text Retrieval", "CLTR", "Cross-Language Information Retrieval", "CLIR",
                            "Multilingual Text Retrieval", "Ad-hoc", "Domain Specific Cross-Language IR", "Interactive Cross-Language IR", "iCLEF", "Spoken Document/Speech Retrieval",
                            "CLEF SR", "Question Answering", "QA@CLEF", "Multimedia Retrieval", "ImageCLEF", "Multilingual Web Search", "WebCLEF", "Geographical Retrieval", 
                            "GeoCLEF", "CLEF@MorphoChallenge", "MorphoChallenge", "Morpho Challenge", "CLEF@SemEval", "SemEval", "Cross-Language Video Retrieval", "VideoCLEF",
                            "Multilingual Information Filtering", "INFILE", "Log File Analysis", "LogCLEF", "Intellectual Property in the Patent Domain", "CLEF-IP",
                            "Component-based Evaluation", "Grid@CLEF", "Web People Search", "WEPS", "Cross-lingual Expert Search", "CriES", "Digital Text Forensics and Stylometry",
                            "PAN", "Music Information Retrieval", "MusiCLEF", "Cultural Heritage in CLEF", "CHiC", "Retrieval on Structured Datasets", "INEX", "Online Reputation Management",
                            "RepLab", "eHealth", "CLEF-ER", "Biomedical Semantic Indexing and Question Answering", "BioASQ", "LifeCLEF", "Biodiversity Identification and Prediction", 
                            "News Recommendation Evaluation", "NewsREEL", "Living Labs", "LL4IR", "Social Book Search", "SBS", "Microblog Cultural Contextualization", "MC2",
                            "Dynamic Search for Complex Tasks", "DynSE", "Multimodal Spatial Role Labeling", "MSRL", "Early Risk Prediction on the Internet", "eRisk", "Personalised Information Retrieval",
                            "PIR-CLEF", "Reproducibility", "CENTRE@CLEF", "Identification and Verification of Political Claims", "CheckThat!", "Extracting Protests from News",
                            "ProtestNews", "Answer Retrieval for Questions on Math", "ARQMath", "ChEMU", "Cheminformatics Elsevier Melbourne University lab", "Identifying Historical People, Places and other Entities",
                            "HIPE", "Living Labs for Academic Search", "LiLAS", "Argument Retrieval", "Touché", "Automatic Simplification of Scientific Texts",
                            "SimpleText", "Intelligent Disease Progression Prediction", "iDPP@CLEF", "Automatic Workplay and Humour Translation", "JokeR", "Learning to Quantify",
                            "LeQua"]
                            
    additional_stopwords = ["TREC", "Track", "CLEF", "Working Notes", "Workshop", "Participation", "Lab"]                       
    def remove_stopwords(text, stopword_list):
    # Create a regular expression pattern to remove all stopwords
        pattern = re.compile(r'\b(?:' + '|'.join(re.escape(word) for word in stopword_list) + r')\b', re.IGNORECASE)
        cleaned_text = pattern.sub('', text)  # Ersetze Stopwords durch leeren String
        return cleaned_text

    text = remove_stopwords(text, stopwords_labs_tracks)
    text = remove_stopwords(text, additional_stopwords)

    words = word_tokenize(text)

    # Stem words
    stemmed_words = [stemmer.stem(word) for word in words]

    stopwords = stopwords.words('english')
    # Remove stopwords
    filtered_words = [word for word in stemmed_words if word not in stopwords]
    
    # Create bi-grams
    bi_grams = [' '.join(bi) for bi in zip(filtered_words, filtered_words[1:])]
    tri_grams = [' '.join(bi) for bi in zip(filtered_words, filtered_words[1:], filtered_words[2:])]

    return filtered_words + bi_grams + tri_grams

In [23]:
vectorizer = TfidfVectorizer(tokenizer=custom_tokenizer)

In [24]:
tfidf_matrix = vectorizer.fit_transform(corpus_text)


In [41]:
import numpy as np
document_idx = 1000
tfidf_values = tfidf_matrix[document_idx].toarray()[0]

# Indizes der TF-IDF-Werte in absteigender Reihenfolge sortieren
sorted_indices = np.argsort(tfidf_values)[::-1]

In [42]:
feature_names = vectorizer.get_feature_names_out()

In [43]:
top_n = 10
for i in range(top_n):
    idx = sorted_indices[i]
    word = feature_names[idx]
    tfidf_value = tfidf_values[idx]
    print(f"Wort: {word}, TF-IDF-Wert: {tfidf_value}")

Wort: expert, TF-IDF-Wert: 0.19245729944626527
Wort: use challeng, TF-IDF-Wert: 0.13720030051956036
Wort: crosslingu expert, TF-IDF-Wert: 0.13062548620234943
Wort: crosslingu expert search, TF-IDF-Wert: 0.13062548620234943
Wort: expert search, TF-IDF-Wert: 0.12596058120574907
Wort: crosslingu, TF-IDF-Wert: 0.11932289869199968
Wort: yahoo, TF-IDF-Wert: 0.11688613849519762
Wort: challeng, TF-IDF-Wert: 0.08396209798404346
Wort: topic, TF-IDF-Wert: 0.07591588010006188
Wort: submit run, TF-IDF-Wert: 0.07297378481078887


In [47]:
corpus_text[3000]

'The speed and scale at which information can be created and propagated has increased markedly over the last few decades and has far exceeded the scope that human fact-checkers can handle, enabling the rise in harmful and compelling disinformation campaigns. As a result, increasing attention is focusing on the "Claim Matching" problem, where a portion of the fact checking process is automated via AI, by matching content with a human verified claims database. In this report, we discuss a novel neural pipeline for claim matching. Specifically, we demonstrate the efficacy of generative re-rankers to aid the claim matching process, and introduce a new training objective that targets maximizing mutual information. In the CLEF CheckThat! 2022 Competition sub-task 2a, our claim matching approach placed first, beating the second place team by over 3.4 percentage points (evaluated on MAP@5).'

## Try extraction with Llama3

In [9]:
import xml.etree.ElementTree as ET

def extract_text_from_abstract(path):
    tree = ET.parse(path)
    root = tree.getroot()
    ns = {'ns0': 'http://www.tei-c.org/ns/1.0'}

    # Find the <abstract> element
    abstract = root.find(".//ns0:abstract", ns)
    
    if abstract is not None:
        # Extract all <p> tags within <div> inside <abstract>
        paragraphs = abstract.findall(".//ns0:div//ns0:p", ns)

        # Extract and concatenate text content from all <p> tags
        extracted_text = "\n".join([p.text.strip() for p in paragraphs if p.text])

        return [path, extracted_text]
    else:
        return None

In [10]:
files = []
for file in glob.glob("../../resources/XML_CEUR/*/*.tei.xml"):
    files.append(file)
for file in glob.glob("../../resources/XML_CEUR/*/*/*.tei.xml"):
    files.append(file)

In [11]:
corpus_text = []

for i in files:
    string = extract_text_from_abstract(i)
    corpus_text.append(string)

In [6]:
import httpx
import json

In [17]:
url = "http://localhost:11434/api/generate" # Current Version 0.3.8

In [28]:
relevant_terms = []

# Classify the sentence with Llama3 and count how many of the citations are referring to the usage of the underlying dataset 

for i in corpus_text[10:20]:
    # Define the prompt for the classification with Llama3

    prompt = """
    Can you identify the underlying technical methods used by the authors in the provided abstract? Focus specifically on techniques such as machine learning models (e.g., Deep Learning, Large Language Models, Support Vector Machines), data processing methods, or other technical approaches. If such methods are mentioned, list them separated by commas. If no relevant technical methods are mentioned, respond with "None". Provide only the methods or "None" without any additional commentary or explanation.
    """
    prompt = prompt + "\nAbstract:\n" + i[1] 
    data = {"model":"llama3", "prompt" : f"{prompt}"}
    response = httpx.post(url, data=json.dumps(data), headers={"Content-Type": "application/json"}, timeout=15)
    response_lines = [line for line in response.text.strip().split("\n") if line]
    response_dicts = [json.loads(line) for line in response_lines]
    answer = "".join(response_dict.get("response", "") for response_dict in response_dicts)
    if answer == "None":
        terms = []
    else:
        if "," in answer:
            terms = [method.strip() for method in answer.split(",")] #  Add multiple found methods to list
        else:
            terms = [answer.strip()]  # Add single method to list
    relevant_terms.append([i[0], i[1], terms])  

In [39]:
relevant_terms[5][1]

'We present an approach to multilingual information retrieval that does not depend on the existence of specific linguistic resources such as stemmers or thesaurii. Using the HAIRCUT system we participated in the monolingual, bilingual, and multilingual tasks of the CLEF-2000 evaluation. Our method, based on combining the benefits of words and character n-grams, was effective for both language-independent monolingual retrieval as well as for cross-language retrieval with translated queries. After describing our monolingual retrieval approach we compare a translation method using aligned parallel corpora to commercial machine translation software.'

In [40]:
relevant_terms[5][2]

['Machine Learning models (None)',
 'Deep Learning (None)',
 'Large Language Models (None)',
 'Support Vector Machines (None)',
 'data processing methods: words',
 'character n-grams.']